In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = yf.download("BSE.NS", start="2022-01-01", end="2025-08-01", actions=True, auto_adjust=False)
df.to_csv("BSE.csv")

In [ ]:
x=pd.read_csv("BSE.csv")
x.head()

In [ ]:
x.describe()

In [ ]:
daily_close = x ["Adj Close"]
daily_return = daily_close.pct_change()
daily_return.fillna(0,inplace=True)
print(daily_return)
daily_return.sum()

In [ ]:
#MAV Calculation
short_mav = x["Adj Close"]. rolling (window=50).mean()
long_mav = x["Adj Close"]. rolling (window=120).mean()
print(short_mav[-10:])

In [ ]:
x["Adj Close"].plot()

In [ ]:
short_mav.plot()
plt.show()
long_mav.plot()

In [ ]:
signal_df = pd.DataFrame(index=x.index)
signal_df ["signal"] = 0.0
signal_df ["short_mav"] = short_mav
signal_df['long_mav']=long_mav
signal_df.loc[50:, "signal"] = np.where(
    signal_df["short_mav"][50:] > signal_df["long_mav"][50:], 
    1.0, 
    0.0
)
signal_df['positions'] = signal_df['signal'].diff()
signal_df[signal_df['positions'] ==-1.0]


In [ ]:
# initialize the plot using plt
fig = plt.figure(figsize=(12,6))
ax=plt.subplot(1,1,1)

# Add a subplot and label for y-axis
x["Adj Close"].plot(ax=ax, lw=2)
signal_df[['short_mav', 'long_mav']].plot(ax=ax, lw=2., figsize=(12,8))
# plotting the sell signals
ax.plot(signal_df.loc[signal_df.positions ==-1.0].index,
 signal_df.short_mav[signal_df.positions ==-1.0],
 'v', markersize=10, color='k')
# plotting the Buy signals
ax.plot(signal_df.loc[signal_df.positions == 1.0].index,
 signal_df.short_mav[signal_df.positions == 1.0],
'^', markersize=10, color='m')

In [ ]:
# =============================================================================
# ADD STOP LOSS AND TAKE PROFIT LOGIC
# =============================================================================

def apply_risk_management(signal_df, daily_return, stop_loss_pct=0.05, take_profit_pct=0.10):
    """
    Apply stop loss and take profit to strategy returns
    """
    # Initialize variables
    strategy_returns = pd.Series(0.0, index=daily_return.index)
    in_position = False
    entry_price = 0
    entry_index = None
    
    # Price data for stop loss calculations
    prices = x['Adj Close']
    
    for i in range(1, len(signal_df)):
        current_signal = signal_df['signal'].iloc[i]
        current_price = prices.iloc[i]
        prev_signal = signal_df['signal'].iloc[i-1]
        
        # ENTRY SIGNAL (Buy)
        if not in_position and current_signal == 1 and prev_signal == 0:
            in_position = True
            entry_price = current_price
            entry_index = i
            continue
        
        # EXIT CONDITIONS (if in position)
        if in_position:
            price_change_pct = (current_price - entry_price) / entry_price
            
            # STOP LOSS HIT
            if price_change_pct <= -stop_loss_pct:
                strategy_returns.iloc[i] = -stop_loss_pct  # Assume stop loss hit
                in_position = False
                entry_price = 0
                entry_index = None
            
            # TAKE PROFIT HIT
            elif price_change_pct >= take_profit_pct:
                strategy_returns.iloc[i] = take_profit_pct  # Assume take profit hit
                in_position = False
                entry_price = 0
                entry_index = None
            
            # NORMAL EXIT (MA crossover sell signal)
            elif current_signal == 0 and prev_signal == 1:
                actual_return = (current_price - entry_price) / entry_price
                strategy_returns.iloc[i] = actual_return
                in_position = False
                entry_price = 0
                entry_index = None
            
            # STILL IN POSITION - calculate intraday return
            else:
                strategy_returns.iloc[i] = daily_return.iloc[i]
        
        # NOT IN POSITION - no return
        else:
            strategy_returns.iloc[i] = 0.0
    
    return strategy_returns




In [ ]:
# =============================================================================
# UPDATED METRICS FUNCTION WITH RISK MANAGEMENT
# =============================================================================

def calculate_strategy_metrics_with_risk_management(x, signal_df, daily_return, 
                                                   stop_loss=0.05, take_profit=0.10, 
                                                   risk_free_rate=0.05):
    """
    Calculate trading strategy performance metrics with stop loss/take profit
    """
    
    # Apply risk management
    strategy_returns = apply_risk_management(signal_df, daily_return, stop_loss, take_profit)
    
    # Remove NaN values
    strategy_returns = strategy_returns.dropna()
    daily_return_clean = daily_return.loc[strategy_returns.index]
    
    # Portfolio values (assuming initial investment of 10000)
    initial_investment = 10000
    strategy_portfolio = initial_investment * (1 + strategy_returns).cumprod()
    buy_hold_portfolio = initial_investment * (1 + daily_return_clean).cumprod()
    
    metrics = {}
    
    # 1. Total Return
    metrics['Strategy Total Return'] = (strategy_portfolio.iloc[-1] / strategy_portfolio.iloc[0]) - 1
    metrics['Buy & Hold Total Return'] = (buy_hold_portfolio.iloc[-1] / buy_hold_portfolio.iloc[0]) - 1
    
    # 2. Sharpe Ratio (Annualized)
    excess_strategy_returns = strategy_returns - risk_free_rate/252
    excess_buy_hold_returns = daily_return_clean - risk_free_rate/252
    
    metrics['Strategy Sharpe Ratio'] = (excess_strategy_returns.mean() / strategy_returns.std() * np.sqrt(252) 
                                        if strategy_returns.std() > 0 else 0)
    metrics['Buy & Hold Sharpe Ratio'] = (excess_buy_hold_returns.mean() / daily_return_clean.std() * np.sqrt(252) 
                                         if daily_return_clean.std() > 0 else 0)
    
    # 3. Portfolio Growth
    metrics['Strategy Portfolio Growth'] = strategy_portfolio.iloc[-1] - strategy_portfolio.iloc[0]
    metrics['Buy & Hold Portfolio Growth'] = buy_hold_portfolio.iloc[-1] - buy_hold_portfolio.iloc[0]
    
    # 4. Risk Management Metrics
    metrics['Stop Loss Used'] = stop_loss
    metrics['Take Profit Used'] = take_profit
    metrics['Stop Loss Hits'] = len(strategy_returns[strategy_returns == -stop_loss])
    metrics['Take Profit Hits'] = len(strategy_returns[strategy_returns == take_profit])
    
    # 5. Win/Loss Metrics
    winning_trades = strategy_returns[strategy_returns != 0]
    if len(winning_trades) > 0:
        metrics['Win Rate'] = (winning_trades > 0).mean()
        metrics['Average Win'] = winning_trades[winning_trades > 0].mean()
        metrics['Average Loss'] = winning_trades[winning_trades < 0].mean()
        metrics['Profit Factor'] = abs(winning_trades[winning_trades > 0].sum() / 
                                      winning_trades[winning_trades < 0].sum())
    else:
        metrics['Win Rate'] = 0
        metrics['Average Win'] = 0
        metrics['Average Loss'] = 0
        metrics['Profit Factor'] = 0
    
    # 6. Max Drawdown
    def calculate_max_drawdown(portfolio_values):
        peak = portfolio_values.expanding().max()
        drawdown = (portfolio_values - peak) / peak
        return drawdown.min()
    
    metrics['Strategy Max Drawdown'] = calculate_max_drawdown(strategy_portfolio)
    metrics['Buy & Hold Max Drawdown'] = calculate_max_drawdown(buy_hold_portfolio)
    
    # 7. Volatility (Annualized)
    metrics['Strategy Volatility'] = strategy_returns.std() * np.sqrt(252)
    metrics['Buy & Hold Volatility'] = daily_return_clean.std() * np.sqrt(252)
    
    # Additional metrics
    metrics['Number of Trades'] = len(signal_df[signal_df['positions'] != 0.0])
    metrics['Number of Buy Signals'] = len(signal_df[signal_df['positions'] == 1.0])
    metrics['Number of Sell Signals'] = len(signal_df[signal_df['positions'] == -1.0])
    
    return metrics, strategy_portfolio, buy_hold_portfolio, strategy_returns

In [ ]:
# =============================================================================
# TEST WITH DIFFERENT RISK PARAMETERS
# =============================================================================

# Test different stop loss and take profit levels
risk_params = [
    (0.03, 0.08),   # Tight stops
    (0.05, 0.10),   # Medium stops  
    (0.08, 0.15),   # Wide stops
    (0.10, 0.20),   # Very wide stops
    (0.12, 0.25),
    (0.15, 0.30),
    (0.20, 0.40)
    
]

best_sharpe = -999
best_params = None
best_metrics = None

print("Testing different risk parameters...")
print("=" * 60)

for stop_loss, take_profit in risk_params:
    try:
        metrics, strategy_portfolio, buy_hold_portfolio, strategy_returns = calculate_strategy_metrics_with_risk_management(
            x, signal_df, daily_return, stop_loss, take_profit
        )
        
        print(f"\nStop Loss: {stop_loss:.1%}, Take Profit: {take_profit:.1%}")
        print(f"Sharpe Ratio: {metrics['Strategy Sharpe Ratio']:.4f}")
        print(f"Total Return: {metrics['Strategy Total Return']:.2%}")
        print(f"Win Rate: {metrics['Win Rate']:.1%}")
        print(f"Stop Loss Hits: {metrics['Stop Loss Hits']}")
        print(f"Take Profit Hits: {metrics['Take Profit Hits']}")
        
        if metrics['Strategy Sharpe Ratio'] > best_sharpe:
            best_sharpe = metrics['Strategy Sharpe Ratio']
            best_params = (stop_loss, take_profit)
            best_metrics = metrics
            
    except Exception as e:
        print(f"Error with SL:{stop_loss}, TP:{take_profit}: {e}")


In [ ]:
# FINAL RESULTS WITH BEST PARAMETERS
print("\n" + "=" * 60)
print("BEST RISK PARAMETERS FOUND")
print("=" * 60)
print(f"Stop Loss: {best_params[0]:.1%}, Take Profit: {best_params[1]:.1%}")
print(f"Best Sharpe Ratio: {best_sharpe:.4f}")

# Run final analysis with best parameters
final_metrics, final_strategy_portfolio, final_buy_hold_portfolio, final_strategy_returns = calculate_strategy_metrics_with_risk_management(
    x, signal_df, daily_return, best_params[0], best_params[1]
)

print("\n" + "=" * 60)
print("FINAL STRATEGY PERFORMANCE WITH RISK MANAGEMENT")
print("=" * 60)

print("\n--- RETURNS ---")
print(f"{'Strategy Total Return':30}: {final_metrics['Strategy Total Return']:>10.3%}")
print(f"{'Buy & Hold Total Return':30}: {final_metrics['Buy & Hold Total Return']:>10.3%}")

print("\n--- RISK METRICS ---")
print(f"{'Strategy Sharpe Ratio':30}: {final_metrics['Strategy Sharpe Ratio']:>10.4f}")
print(f"{'Strategy Max Drawdown':30}: {final_metrics['Strategy Max Drawdown']:>10.3%}")
print(f"{'Strategy Volatility':30}: {final_metrics['Strategy Volatility']:>10.3%}")

print("\n--- RISK MANAGEMENT STATS ---")
print(f"{'Stop Loss Hits':30}: {final_metrics['Stop Loss Hits']:>10}")
print(f"{'Take Profit Hits':30}: {final_metrics['Take Profit Hits']:>10}")
print(f"{'Win Rate':30}: {final_metrics['Win Rate']:>10.2%}")
print(f"{'Average Win':30}: {final_metrics['Average Win']:>10.3%}")
print(f"{'Average Loss':30}: {final_metrics['Average Loss']:>10.3%}")
print(f"{'Profit Factor':30}: {final_metrics['Profit Factor']:>10.3f}")
